# This notebook will run through the general fitting process for matching CO models to late-time spectra.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib
from scipy.ndimage import gaussian_filter1d
import os

import get_model

In [11]:
path = os.getcwd()
co_path = '/'.join(path.split('/')[:-1])+'/'
plot = 0
coplus = 0

In [28]:
redshift  = 0.002
specFile = 'SN2013ej_20131214_fire.dat'
specPhase = '143.8'

# This is all just reading the spectrum to fit

In [29]:
spec=pd.read_csv(specFile,delimiter='\s+',header=None)
    
if spec.shape[1] == 2:
    spec.columns = ['lambda','flux']  
if spec.shape[1] == 3:   
    spec.columns = ['lambda','flux','err']
if spec['lambda'].values[10] > 1000:
    spec['lambda'] = spec['lambda']/1e4
spec['lambda'] = spec['lambda']/(1+redshift)
spec = spec[(spec['lambda']>2.2)]

# data scaling
upper, lower = 2.20, 2.25
if coplus: upper, lower = 2.20, 2.25
w = np.argmin(abs(np.array(spec['lambda'])-upper))
v = np.argmin(abs(np.array(spec['lambda'])-lower))
integ0 = np.trapz(spec['flux'][w:v],spec['lambda'][w:v])
off = 0.05/integ0
q = [(spec['lambda'] >= 2.25) & (spec['lambda'] <= 2.3)][0]
spec['flux'] = np.array(spec['flux'])*off
spec['flux'] = spec['flux'] - (1.0 - np.trapz(spec['flux'][q],spec['lambda'][q]))
spec = spec[(spec['flux'] > 0)]

# For speed within this Python script we want to use dataframes and csv files. We can use the raw model output files but they are about 10 times slower to read.

In [32]:
df = pd.read_csv(co_path+'Hoeflich_models.csv')

# Limiting to only the models that we care about for this example, i.e. what will fit reasonably well.
df = df[(df.molecule == 'CO')]
df = df[(df.mole_frac==0.0)&(df.velocity<2000.)] 
df

,filename,dataset,mole_frac,temperature,density,velocity,molecule
256,plot.dat_500_4_0.0_CO,4,0.0,5000.0,1.000000e+07,500,CO
257,plot.dat_500_4_0.0_CO,4,0.0,5000.0,1.000000e+08,500,CO
258,plot.dat_500_4_0.0_CO,4,0.0,5000.0,1.000000e+09,500,CO
259,plot.dat_500_4_0.0_CO,4,0.0,5000.0,1.000000e+11,500,CO
260,plot.dat_500_4_0.0_CO,4,0.0,4000.0,1.000000e+07,500,CO
...,...,...,...,...,...,...,...
2203,plot.dat_500_3_0.0_CO,3,0.0,1750.0,1.000000e+11,500,CO
2204,plot.dat_500_3_0.0_CO,3,0.0,1500.0,1.000000e+07,500,CO
2205,plot.dat_500_3_0.0_CO,3,0.0,1500.0,1.000000e+08,500,CO
2206,plot.dat_500_3_0.0_CO,3,0.0,1500.0,1.000000e+09,500,CO


In [ ]:
# this just supresses all the warnings that numpy and python like to throw
import warnings
warnings.filterwarnings("ignore")

def get_model_new(path,temp,dens,velocity,mole_frac,molecule,modelset):
    modeldf = pd.read_csv(co_path+'/models_csv/'+
                          molecule+'_'+str(modelset)+'_'+str(velocity)+'_'+str(mole_frac)+'.csv')

    dens = str(format(dens, "10.2E")).strip()
    temp = str(format(temp, ".2f")).strip()

    return [modeldf['wl'].values, modeldf[dens+'/'+temp].values]

# loop over models
chisq, fArr, fileArr, params = [], [], [], []
for i in range(len(df)):
    if i%100 == 0 and i != 0: 
        print(str(round(i/len(df)*100,1))+'% done.')
        
    # get model from csv file
    wav, opac = get_model.get_csv(co_path+'/models_csv/',df.temperature.values[i],
                                  df.density.values[i],df.velocity.values[i],
                                  df.mole_frac.values[i],df.molecule.values[i],df.dataset.values[i])
    wav = wav/1e4
    
    # trim model
    w = np.where(wav >= 2.2)
    wav, opac = np.array(wav[w]), np.array(opac[w])
    
    # model scaling
    w = np.argmin(abs(wav-upper))
    v = np.argmin(abs(wav-lower))
    q = [(spec['lambda'] >= 2.25) & (spec['lambda'] <= 2.3)][0]
    integ = np.min(spec['flux'][q]) + np.trapz(opac[w:v],wav[w:v])
    opac = opac + integ
    interpF = np.interp(spec['lambda'],wav,opac) # interpolating to same x
    # scaling the model to match the data since the amount of CO is a free parameter
    w = [(spec['lambda'] >= 2.31)&(spec['lambda'] <= 2.33)][0]
    ispec, imodel = np.trapz(spec['flux'][w],spec['lambda'][w]), np.trapz(interpF[w],spec['lambda'][w])
    interpF = interpF*ispec/imodel
    
    if plot:
        plt.plot(spec['lambda'],spec['flux'],color='black')
        plt.plot(spec['lambda'],interpF,color='red')
        plt.show()
        
    w = [(spec['lambda'] >= 2.28) & (spec['lambda'] <= 2.46)][0]
    chisq.append(stats.chisquare(interpF[w],spec['flux'].values[w])[0])
    fArr.append(interpF)
    fileArr.append(i)
    params.append([df.temperature.values[i],df.density.values[i],df.velocity.values[i],\
                   df.mole_frac.values[i],df.dataset.values[i]])
    
w = np.argsort(chisq)
chisq, fArr, fileArr, params = np.array(chisq)[w], np.array(fArr)[w], np.array(fileArr)[w], np.array(params)[w]

# plot best XX models for each spectrum
matplotlib.rcParams.update({'font.size': 18})
for i in range(25):
    label = r'T: '+str(params[i][0])+'\n'+'n: '+format(params[i][1],".2E")+'\n'+\
             'velocity: '+str(params[i][2])+'\n'+'frac: '+str(params[i][3])+'\n'+'model: '+str(params[i][4])
    
    w = [(spec['lambda'].values < 2.5)]
    plt.plot(spec['lambda'].values[w],spec['flux'].values[w],color='black')
    plt.plot(spec['lambda'].values[w],fArr[i][w],color='red',label=label)
    
    q = [(spec['lambda'].values > 2.2) & (spec['lambda'].values < 2.5)]
    plt.xlabel(r'Wavelength ($\mu m$)')
    plt.ylabel('Scaled Flux')
    plt.legend(bbox_to_anchor=(1.0,0.5), loc="center left",labels=[str(specPhase)+'d',label])

    plt.show()